In [8]:
import os
import openai
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

In [4]:
# pip install -q google.generativeai

In [9]:
import os
import google.generativeai as palm
from google.api_core import client_options as client_options_lib

palm.configure(
    api_key=os.getenv("PALM_API_KEY"),
    transport="rest",
    client_options=client_options_lib.ClientOptions(
        api_endpoint=os.getenv("GOOGLE_API_BASE"),
    )
)

### Explore the available models

In [10]:
for m in palm.list_models():
    print(f"name: {m.name}")
    print(f"description: {m.description}")
    print(f"generation methods:{m.supported_generation_methods}\n")

name: models/chat-bison-001
description: Chat-optimized generative language model.
generation methods:['generateMessage', 'countMessageTokens']

name: models/text-bison-001
description: Model targeted for text generation.
generation methods:['generateText', 'countTextTokens', 'createTunedTextModel']

name: models/embedding-gecko-001
description: Obtain a distributed representation of a text.
generation methods:['embedText']



#### Filter models by their supported generation methods
- `generateText` is currently recommended for coding-related prompts.
- `generateMessage` is optimized for multi-turn chats (dialogues) with an LLM.`

In [12]:
models = [m for m in palm.list_models() 
          if 'generateText' 
          in m.supported_generation_methods]
print(models)

[Model(name='models/text-bison-001', base_model_id='', version='001', display_name='Text Bison', description='Model targeted for text generation.', input_token_limit=8196, output_token_limit=1024, supported_generation_methods=['generateText', 'countTextTokens', 'createTunedTextModel'], temperature=0.7, top_p=0.95, top_k=40)]


In [13]:
model_bison = models[0]
model_bison

Model(name='models/text-bison-001', base_model_id='', version='001', display_name='Text Bison', description='Model targeted for text generation.', input_token_limit=8196, output_token_limit=1024, supported_generation_methods=['generateText', 'countTextTokens', 'createTunedTextModel'], temperature=0.7, top_p=0.95, top_k=40)

#### helper function to generate text

- The `@retry` decorator helps you to retry the API call if it fails.
- We set the temperature to 0.0 so that the model returns the same output (completion) if given the same input (the prompt).

In [14]:
from google.api_core import retry
@retry.Retry()
def generate_text(prompt,
                  model=model_bison,
                  temperature=0.0):
    return palm.generate_text(prompt=prompt,
                              model=model,
                              temperature=temperature)

#### Ask the LLM how to write some code


In [25]:
prompt = "Show me how to write a Python class that would connect to a SQLite database, create tables, and do all of the CRUD operations on the data.  Also, I'd like to see examples of how to call the functions of that class."
completion = generate_text(prompt)
print(completion.result)

```python
import sqlite3

class SQLiteDB:

    def __init__(self, db_file):
        """Create a connection to the SQLite database file.

        Args:
            db_file (str): The path to the SQLite database file.
        """
        self.conn = sqlite3.connect(db_file)
        self.cur = self.conn.cursor()

    def create_table(self, table_name, columns):
        """Create a table in the database.

        Args:
            table_name (str): The name of the table to create.
            columns (list): A list of tuples, where each tuple contains the name and type of a column.
        """
        sql = """CREATE TABLE {} ({})""".format(
            table_name,
            ', '.join('{} {}'.format(col[0], col[1]) for col in columns)
        )
        self.cur.execute(sql)
        self.conn.commit()

    def insert_row(self, table_name, values):
        """Insert a row into a table.

        Args:
            table_name (str): The name of the table to insert into.
            values (li

#### Try out the code
- Try copy-pasting some of the generated code and running it in the notebook.
- Remember to test out the LLM-generated code and debug it make sure it works as intended.

In [23]:
# Create a list
list1 = ["apple", "banana", "cherry"]

# Iterate over the list using a for loop
for item in list1:
    print(item)

apple
banana
cherry
